In [ ]:
!pip install geopandas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import plotly.express as px


**Este proyecto es un Análisis Exploratorio de Datos (EDA) y limpieza de datos usando un dataset de airbnb ubicado en la CDMX. El objetivo principal es ayudar a un inversor ficticio a decidir dónde comprar una propiedad para alquilarla en Airbnb.**

*con df.head mostramos las primeras 5 filas para una inspección inicial y df.info resume las columnas, tipo de datos y valores no nulos.*

In [ ]:
df = pd.read_csv('listings.csv')
print(df.head())
print(df.info())

      id                                              name  host_id host_name  \
0  35797                                       Villa Dante   153786      Dici   
1  44616                                      Condesa Haus   196253  Fernando   
2  56074              Great space in historical San Rafael   265650     Maris   
3  67703                 2 bedroom apt. deco bldg, Condesa   334451  Nicholas   
4  70644  Beautiful light Studio Coyoacan- full equipped !   212109    Trisha   

   neighbourhood_group          neighbourhood  latitude  longitude  \
0                  NaN  Cuajimalpa de Morelos  19.38283  -99.27178   
1                  NaN             Cuauhtémoc  19.41162  -99.17794   
2                  NaN             Cuauhtémoc  19.43977  -99.15605   
3                  NaN             Cuauhtémoc  19.41152  -99.16857   
4                  NaN               Coyoacán  19.35448  -99.16217   

         room_type    price  minimum_nights  number_of_reviews last_review  \
0  Entire home

In [ ]:
print("\n--- NOMBRES DE COLUMNAS REALES (LISTA COMPLETA) ---")
print(df.columns.tolist())


--- NOMBRES DE COLUMNAS REALES (LISTA COMPLETA) ---
['id', 'name', 'host_id', 'host_name', 'neighbourhood_group', 'neighbourhood', 'latitude', 'longitude', 'room_type', 'price', 'minimum_nights', 'number_of_reviews', 'last_review', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365', 'number_of_reviews_ltm', 'license']


Primero definimos las columnas que usaremos para el EDA enfocandonos en el precio, ubicación y actividad de reseñas. Crearemos el data frame final a partir del data frame original con las columnas ya organizadas.

In [ ]:
columnas_eda = ['price', 'latitude', 'longitude', 'neighbourhood_group',
                'neighbourhood', 'room_type', 'minimum_nights', 'number_of_reviews',
                'reviews_per_month', 'availability_365']

df_final = df[columnas_eda].copy()

print(f"df_final creado con {len(df_final.columns)} columnas y {len(df_final)} filas.")

df_final creado con 10 columnas y 26401 filas.


**Limpieza (Data Cleaning) y Preprocesamiento**

Primero veremos el porcentaje de NaNs que hay en las columnas críticas o importantes que serían price y reviews_per_month. Le indicamos las columnas que necesitamos. Obtenemos el total de las filas para usarlo como denominador usando len(df_final). Contamos el número de NaNs para las columnas criticas usando .isnull().sum() sobre el objeto creado columnas_criticas. Y calculamos el porcentaje dividiendo el conteo de las columnas NaN con el total de filas y multiplicando por 100. Usamos apply para aplicar la función y formatear los valores a dos decimales en el porcentaje.

In [ ]:

columnas_criticas = ['price', 'reviews_per_month']

total_filas = len(df_final)

nan_count = df_final[columnas_criticas].isnull().sum()

porcentaje_nan = (nan_count / total_filas) * 100

print("--- Porcentaje de Valores Faltantes (NaN) en Columnas Críticas ---")
print(f"Total de filas iniciales: {total_filas}")
print("\nPorcentaje de NaN por Columna:")
print(porcentaje_nan.apply(lambda x: f"{x:.2f}%")) # Formateamos a dos decimales

--- Porcentaje de Valores Faltantes (NaN) en Columnas Críticas ---
Total de filas iniciales: 26401

Porcentaje de NaN por Columna:
price                12.40%
reviews_per_month    12.78%
dtype: object


Ahora limpiamos la columna price eliminando simbolos de moneda y comas para asegurar que la columna sea numérica. Usaremos .astype(str) para convertirlo en string y str.replace para reemplazar ambos simbolos. usaremos astype(float) para asegurarnos que nos arroje un valor y no un objeto.

In [ ]:
df_final['price'] = df_final['price'].astype(str) \
                                     .str.replace('$', '', regex=False) \
                                     .str.replace(',', '', regex=False) \
                                     .astype(float)

Ahora identificaremos los NaNs en las columnas importantes como price y  reviews_per_month. Usaremos el df_final e indicaremos con que haga la suma de las filas donde los valores son NaN con .isnull().sum()

In [ ]:
print("\n--- Conteo de NaN en columnas clave ANTES de la limpieza ---")
print(df_final[['price', 'reviews_per_month']].isnull().sum())


--- Conteo de NaN en columnas clave ANTES de la limpieza ---
price                3274
reviews_per_month    3373
dtype: int64


In [ ]:
print("Conteo de NaN en neighbourhood_group:")
print(df_final['neighbourhood'].isnull().sum())

Conteo de NaN en neighbourhood_group:
0


Para la limpieza de los datos eliminaremos las filas NaN en price ya que estos precios podrían sesgar nuestros calculos. Definimos la cantidad de filas que hay antes de hacer dropna. Y eliminamos las filas NaN con df_final.dropna(subset=['price']), usamos subset para elegir la columna especifica. Para reviews_per_month lo imputaremos con "0" ya que las que tienen valores NaN pueden ser solo viviendas nuevas por lo tanto no tienen reseñas, para eso usaremos .fillna(0). Verificamos cual es el conteo de las NaN después de la limpieza.

In [ ]:

filas_iniciales = len(df_final)

df_final = df_final.dropna(subset=['price'])

df_final['reviews_per_month'] = df_final['reviews_per_month'].fillna(0)

print("\n--- Conteo de NaN en df_final DESPUÉS de la limpieza ---")
print(df_final[['price', 'reviews_per_month']].isnull().sum())


--- Conteo de NaN en df_final DESPUÉS de la limpieza ---
price                0
reviews_per_month    0
dtype: int64


Ahora identificaremos y filtraremos los precios inusualmente altos o bajos que podrían ser errores de entrada o listados de lujo extremo. Usaremos la técnica del Rango Intercuartílico (IQR). Primero hacemos una visualización de precios antes del filtrado de outliers usando un boxplot para identificar valores extremos.

In [ ]:
fig_boxplot_simple = px.box(
    df_final,
    y='price',


    title='Distribución de Precio (Antes de Outliers) - Interactivo',
    labels={
        'price': 'Precio por Noche (USD)'
    },
    height=500# Altura de la figura
)


fig_boxplot_simple.update_layout(
    xaxis={'visible': False, 'showticklabels': False}
)

fig_boxplot_simple.show()

Ahora almacenamos el número de filas antes de filtrar usando len y guardandolo en filas_antes_outliers. Después calculamos Q1, Q2 y el IQR. Q1 sería el 25% de los datos y Q3 el 75% cualquier cifra que supere el Q3 * 1.5 * IQR será eliminada. Solo se definirá el límite superior ya que todos los datos son > 0.

In [ ]:
filas_antes_outliers = len(df_final)

Q1 = df_final['price'].quantile(0.25)
Q3 = df_final['price'].quantile(0.75)
IQR = Q3 - Q1

limite_superior = Q3 + 1.5 * IQR

df_final = df_final[df_final['price'] <= limite_superior]

In [ ]:
filas_eliminadas = filas_antes_outliers - len(df_final)
print(f"Precios superiores al límite de ${limite_superior:.2f} eliminados.")
print(f"Filas eliminadas por Outliers: {filas_eliminadas}")
print(f"df_final tiene ahora {len(df_final)} filas.")

Precios superiores al límite de $2621.00 eliminados.
Filas eliminadas por Outliers: 174
df_final tiene ahora 20635 filas.


Ahora mostramos estadísticas descriptivas finales después de eliminar los outliers.

In [ ]:
print("\nEstadísticas Descriptivas Finales del Precio:")
print(df_final['price'].describe())


Estadísticas Descriptivas Finales del Precio:
count    20809.000000
mean      1059.730742
std        584.385723
min         66.000000
25%        591.000000
50%        950.000000
75%       1403.000000
max       2725.000000
Name: price, dtype: float64


Ahora usaremos la estadística descriptiva y la visualización para extraer información procesable. La pregunta clave ahora es ¿Que factores influyen mas en el precio?

Usando la columna room_type y price haremos una comparativa con velas para calcular el promedio en los diferentes tipos de habitación para observar cual de ellas es la que genera más ingresos.

In [ ]:
fig_boxplot = px.box(
    df_final,
    x='room_type',
    y='price',
    color='room_type',

    color_discrete_sequence=px.colors.qualitative.Vivid,

    title='Distribución de Precio por Tipo de Habitación (Interactivo)',
    labels={
        'room_type': 'Tipo de Habitación',
        'price': 'Precio por Noche (USD)'
    },
    hover_data=['neighbourhood']
)


fig_boxplot.update_layout(
    xaxis={'categoryorder': 'total descending'},
)

fig_boxplot.show()

Ahora veremos el precio promedio de cada barrio o vecindario, usaremos groupby para agrupar todos los que tengan el mismo valor como "Centro", "Norte" por ejemplo y con .mean() aplicado a "price" obtendremos el promedio por cada grupo y usando .sort_values(ascending=false) haremos que ordene estos resultados de mayor a menor de manera descendiente. Y renombramos las columnas para usar en plotly.

In [ ]:

precios_por_barrio = df_final.groupby('neighbourhood')['price'].mean() \
                                     .sort_values(ascending=False)

df_top_10_barrios = precios_por_barrio.head(10).reset_index()

df_top_10_barrios.columns = ['Barrio', 'Precio Promedio']

In [ ]:
fig_barras = px.bar(
    df_top_10_barrios,
    x='Barrio',
    y='Precio Promedio',

    color='Precio Promedio',
    color_continuous_scale=px.colors.sequential.Plasma,

    title='Top 10 Barrios con el Precio Promedio Más Alto',
    labels={
        'Barrio': 'Barrio (Neighbourhood)',
        'Precio Promedio': 'Precio Promedio por Noche (USD)'
    },
    height=500
)

fig_barras.update_layout(xaxis={'tickangle': 45})

fig_barras.show()

Para continuar con el analisis contestaremos la pregunta ¿Qué zonas ofrecen mayor rentabilidad potencial? Para esto buscaremos propiedades que cumplan dos condiciones, una alta demanda (reviews por mes) y precios por debajo de la media de la ciudad.

Calculamos la media de los precios y la media de reviews que hay por mes en toda la ciudad usando .mean

In [ ]:
# 1. Calcular las medias de la ciudad (benchmarks)
precio_medio_ciudad = df_final['price'].mean()
reviews_media_ciudad = df_final['reviews_per_month'].mean()

print(f"Precio Promedio Ciudad: ${precio_medio_ciudad:.2f}")
print(f"Reviews Promedio Ciudad: {reviews_media_ciudad:.2f} reviews/mes\n")

Precio Promedio Ciudad: $1059.73
Reviews Promedio Ciudad: 1.75 reviews/mes



ahora calculamos promedios por barrio, usamos groupby para agrupar cada barrio y usamos .agg para agregar dos funciones, una para el precio usando mean y otra para las reviews igual usando mean. usamos .reset_index para transformar los datos.

In [ ]:
# 2. Calcular promedios por barrio ('neighbourhood')
df_barrios_actividad = df_final.groupby('neighbourhood').agg(
    precio_medio=('price', 'mean'),
    reviews_promedio=('reviews_per_month', 'mean')
).reset_index()

ahora vamos a filtrar las oportunidades usando un filtro booleano donde el precio es mas bajo que la media y las reviews mas altas que la media y esos datos los ordenamos con sort_values para que nos muestre de manera descendiente las propiedades que ya cumplen los criterios anteriores y los ordene con los que mas demanda tienen y generamos un top 10

In [ ]:
oportunidades = df_barrios_actividad[
    (df_barrios_actividad['precio_medio'] < precio_medio_ciudad) &
    (df_barrios_actividad['reviews_promedio'] > reviews_media_ciudad)
].sort_values(by='reviews_promedio', ascending=False)

top_10_oportunidades = oportunidades.head(10)

print("\n🚨 Top 10 Oportunidades (Alta Actividad / Precio Bajo):")
print(top_10_oportunidades)


🚨 Top 10 Oportunidades (Alta Actividad / Precio Bajo):
          neighbourhood  precio_medio  reviews_promedio
13  Venustiano Carranza    746.997442          2.582762
0          Azcapotzalco    670.768546          1.993501
6             Iztacalco    814.639506          1.800173


Ahora usaremos plotly para visualizar las propiedades según su precio y otra para visualizar según su demanda.

Primero filtramos los 10 barrios mas caros usamos groupby y mean ordenamos en descendiente y creamos el nuevo data frame para usarlo en la visualización con plotly.

In [ ]:
barrios_mas_caros_nombres = df_final.groupby('neighbourhood')['price'].mean() \
                                     .sort_values(ascending=False).head(10).index.tolist()

df_top_10_caros = df_final[df_final['neighbourhood'].isin(barrios_mas_caros_nombres)].copy()

print(f"Número total de listados en estos barrios: {len(df_top_10_caros)}")

Número total de listados en estos barrios: 18882


In [ ]:
LAT_CENTRO = 19.43
LON_CENTRO = -99.13

fig_top_10_color = px.scatter_mapbox(
    df_top_10_caros,
    lat="latitude",
    lon="longitude",


    color="neighbourhood",


    size="price",
    size_max=10,

    hover_name="neighbourhood",
    hover_data={'room_type': True, 'price': '$,.2f'},

    zoom=9.5,
    center={"lat": LAT_CENTRO, "lon": LON_CENTRO},
    opacity=0.8,
    height=600,
    title="Mapa: Listados de los 10 Barrios Más Caros (Color por Barrio)"
)

fig_top_10_color.update_layout(mapbox_style="carto-positron")
fig_top_10_color.update_layout(margin={"r":0,"t":40,"l":0,"b":0})

fig_top_10_color.show()

después hacemos lo mismo con la demanda

In [ ]:
demanda_por_barrio = df_final.groupby('neighbourhood')['reviews_per_month'].mean() \
                                     .sort_values(ascending=False).head(10).index.tolist()

df_final['Categoria_Demanda'] = df_final['neighbourhood'].apply(
    lambda x: x if x in demanda_por_barrio else 'Resto de Barrios'
)

In [ ]:

LAT_CENTRO = 19.43
LON_CENTRO = -99.13

fig_filtro = px.scatter_mapbox(
    df_final,
    lat="latitude",
    lon="longitude",

    color="Categoria_Demanda",

    size="reviews_per_month",
    size_max=10,

    hover_data=['neighbourhood', 'reviews_per_month'],

    zoom=9.5,
    center={"lat": LAT_CENTRO, "lon": LON_CENTRO},
    opacity=0.7,
    height=700,
    title='Mapa Interactivo: Listados por Categoría de Demanda (Top 10 Barrios)'
)

fig_filtro.update_layout(mapbox_style="carto-positron")
fig_filtro.update_layout(margin={"r":0,"t":40,"l":0,"b":0})

fig_filtro.show()

#  Conclusión y Recomendaciones de Inversión

Basado en el Análisis Exploratorio de Datos (EDA) del mercado de Airbnb, hemos identificado los motores de precio y las oportunidades con el **mejor equilibrio entre demanda y costo**.

## 1. Factores Clave del Precio y Ubicación

Los factores más influyentes para maximizar los ingresos por noche son:

* **Tipo de Propiedad (Room Type):** El listado más rentable es consistentemente **Entire home/apt**, con un precio promedio de **$1,173**.
* **Ubicación Premium:** El barrio con el precio promedio más alto es **Cuajimalpa de Morelos**. Los mapas de Plotly confirman que esta zona forma un *cluster* de **precios altos** en el mapa de calor.

## 2. Oportunidades de Inversión

El análisis se centró en encontrar barrios que operan **por debajo de la media de precio** pero **por encima de la media de demanda (reviews)**.

* **Contexto del Mercado:**
    * Precio Promedio de la Ciudad: **$1059**
    * Actividad Promedio (Demanda): **1.25** reviews/mes

* **Recomendación Central:** Se identificaron **3** barrios que cumplen estos criterios estrictos. La mejor oportunidad de inversión es **Venustiano Carranza**, ya que presenta la **mayor demanda** entre las opciones competitivas.
    * **Métricas Clave:** Precio Promedio de **$746.99** vs. Demanda de **2.5** reviews/mes.

* **Validación Geoespacial:** El mapa de calor de Plotly para la demanda muestra que **Cuauhtemoc** es un **punto caliente de actividad** , confirmando que esta zona es muy popular entre los viajeros a pesar de su precio competitivo.

## 3. Estrategia Final para el Inversor

Se aconseja al inversor la siguiente estrategia de inversión:

1.  **Priorizar el Tipo de Propiedad:** Invertir en listados de tipo **Entire Home**.
2.  **Enfoque Geográfico:** Concentrar la búsqueda de compra en el barrio **Venustiano Carranza**.
3.  **Resultado Esperado:** Al combinar una propiedad de alto valor con una zona de alta demanda y bajo costo de entrada, se maximiza la ocupación y el retorno de la inversión.